# Shunting Yard Algorithm

convert infix to postfix


In [219]:
import re

def solve_or_in_brackets(regex):
    # loop over the string to add | between any two consecutive characters inside square brackets
    i = 0
    regex = list(regex)
    while(i < len(regex)):
        if regex[i] == '[':
            while(regex[i] != ']'):
                if regex[i].isalnum() and regex[i+1].isalnum():
                    regex.insert(i+1, '|')
                i += 1
        else :
            i = i + 1
    return ''.join(regex)


def solve_ranges_in_brackets(regex):
    # loop over the string to add ranges of characters inside square brackets
    i = 0
    regex = list(regex)
    while(i < len(regex)):
        if regex[i] == '[':
            regex[i] = '('
            while(regex[i] != ']'):
                if regex[i] == '-':
                    first = regex[i-1]
                    last = regex[i+1]
                    chars_between = [chr(i) for i in range(ord(first) + 1, ord(last))]
                    str = []
                    for c in chars_between:
                        str.append("|")
                        str.append(c)
                    str.append('|')
                    regex[i:i+1] = str
                    print(''.join(regex))
                i += 1
            regex[i] = ')'
        else :
            i = i + 1
    return ''.join(regex)

def replace_square(regex):
    regex = solve_or_in_brackets(regex)
    regex = solve_ranges_in_brackets(regex)
    return regex

# def replace_square(regex):
#     i = 0
#     regex = list(regex)
#     while(i < len(regex)):
#         if regex[i] == '[':
#             regex[i] = '('
#             while(regex[i] != ']'):
#                 if regex[i] == '-':
#                     first = regex[i-1]
#                     last = regex[i+1]
#                     chars_between = [chr(i) for i in range(ord(first) + 1, ord(last)+1)]
#                     str = []
#                     for c in chars_between:
#                         str.append("|")
#                         str.append(c)
#                     str.append('|')
#                     regex[i:i+2] = str
#                 else : 
#                     # print(regex[i],regex[i+1])
#                     if regex[i] != '(' and regex[i] != '|' and regex[i+1] != '|' and regex[i+1] != '-': 
#                         regex.insert(i+1, '|')
#                 i += 1
#             regex[i-1:i+1] = ')'
#         else :
#             i = i + 1
#     return ''.join(regex)

def replace_dot(regex):
    result = []
    regex = ''.join(regex)
    
    for curr in regex:
        if curr == '.':
            result.append("[a-zA-Z0-9]")
        else : 
            result.append(curr)
    
    return ''.join(result)


def add_concat(regex):
    result = []
    prev = None

    for curr in regex:
        if prev and (prev.isalnum() or prev in ('*', '+', ')','?')) and (curr.isalnum() or curr == '(' ):
            result.append('·')  
        result.append(curr)
        prev = curr
    return ''.join(result)

def preprocessing(regex):

    regex = replace_dot(regex)
    print("\nRegex after replacing dot  : ")
    print(regex)
    regex = replace_square(regex)
    print("\nRegex after replacing square brackets  : ")
    print(regex)
    # regex = add_concat(regex)
    print("\nRegex after adding concatination  : ")
    print(regex)    
    return regex



def infix_to_postfix(infix):
    try:
        re.compile(infix)
    except re.error:
        print("Invalid regex")
        return None
    # Add implicit concatenation operator ('.')
    output = []
    for i, char in enumerate(infix):
        output.append(char)
        
        if i < len(infix) - 1:
            next_char = infix[i + 1]
            if char not in "(|." and next_char not in ")*+?|)":
                output.append('.')
    infix = output

    precedence = { '*' : 5, '+' : 4, '?' : 3 , '.' : 2 , '|' : 1 , '(' : 0 }

    infix = list(infix)
    stack = []
    postfix = []
    for i,char in enumerate(infix):
        
        if char == '(':
            stack.append(char)
        
        elif char == ")":
            stack_top = stack.pop()
            while stack_top != "(":
                postfix.append(stack_top)
                stack_top = stack.pop()
        
        elif char in ['*','+','?','.','|']:
            stack_top = None
            if len(stack) > 0:
                stack_top = stack.pop()
            while stack_top != None  and precedence[char] <= precedence[stack_top]:
                postfix.append(stack_top)
                if len(stack) != 0:
                    stack_top = stack.pop()
                else :
                    break
            if stack_top != None and precedence[char] > precedence[stack_top]:
                stack.append(stack_top)
            stack.append(char)
        
        else:
            postfix.append(char)
                        
    
    while stack:
        top = stack.pop()
        if top == '(':
            print("Unbalanced parentheses")
            return None
        postfix.append(top)
    
    return ''.join(postfix)


infix = "[a-g]?o+"
infix="(a*)*"
preprocessed_infix = preprocessing(infix)
print("\nPOSTFIX :")
print(infix_to_postfix(preprocessed_infix))



Regex after replacing dot  : 
(a*)*

Regex after replacing square brackets  : 
(a*)*

Regex after adding concatination  : 
(a*)*

POSTFIX :
a**


# Postfix to nfa

In [229]:
import json

class NFA:
    def __init__(self, states, starting_state=None, accepting_states=[]):
        self.states = states
        self.starting_state = starting_state
        self.accepting_states = accepting_states

class State1:
    def __init__(self, name, accepting=False):
        self.name = name
        self.transitions = {}  # Dictionary: {symbol: [target_states]}
        self.accepting = accepting

    def add_transition(self, symbol, target_state):
        if symbol not in self.transitions:
            self.transitions[symbol] = []
        self.transitions[symbol].append(target_state)

def new_state(state_counter, accepting=False):
    state = State1(f"S{state_counter}", accepting)
    return state

def handle_character(stack, state_counter, symbol):
    start = new_state(state_counter)
    end = new_state(state_counter + 1, True)
    start.add_transition(symbol, end)
    stack.append(NFA([start, end], start, [end]))
    return state_counter + 2

def handle_concatenation(stack, state_counter):
    if len(stack) < 2:
        raise ValueError("Invalid postfix expression for concatenation")
    nfa2 = stack.pop()
    nfa1 = stack.pop()
    for accepting_state in nfa1.accepting_states:
        accepting_state.accepting = False
        accepting_state.add_transition('ε', nfa2.starting_state)
    combined_nfa = NFA(nfa1.states + nfa2.states, nfa1.starting_state, nfa2.accepting_states)
    stack.append(combined_nfa)
    return combined_nfa,state_counter

def handle_or(stack, state_counter):
    nfa2 = stack.pop()
    nfa1 = stack.pop()
    start = new_state(state_counter)
    end = new_state(state_counter + 1, True)

    start.add_transition('ε', nfa1.starting_state)
    start.add_transition('ε', nfa2.starting_state)

    for accepting_state in nfa1.accepting_states + nfa2.accepting_states:
        accepting_state.accepting = False
        accepting_state.add_transition('ε', end)

    combined_nfa = NFA(nfa1.states + nfa2.states + [start, end], start, [end])
    stack.append(combined_nfa)
    return state_counter + 2

def handle_kleene_star(stack, state_counter):
    nfa = stack.pop()
    start = new_state(state_counter)
    end = new_state(state_counter + 1, True)

    start.add_transition('ε', nfa.starting_state)
    start.add_transition('ε', end)

    for accepting_state in nfa.accepting_states:
        accepting_state.accepting = False
        accepting_state.add_transition('ε', end)
        accepting_state.add_transition('ε', start)

    combined_nfa = NFA(nfa.states + [start, end], start, [end])
    stack.append(combined_nfa)
    return state_counter + 2

def handle_one_or_more(stack, state_counter):
    nfa = stack.pop()
    start = new_state(state_counter)
    end = new_state(state_counter + 1, True)

    start.add_transition('ε', nfa.starting_state)

    for accepting_state in nfa.accepting_states:
        accepting_state.accepting = False
        accepting_state.add_transition('ε', end)
        accepting_state.add_transition('ε', nfa.starting_state)

    combined_nfa = NFA(nfa.states + [start, end], start, [end])
    stack.append(combined_nfa)
    return state_counter + 2

def handle_zero_or_one(stack, state_counter):
    nfa = stack.pop()
    start = new_state(state_counter)
    end = new_state(state_counter + 1, True)

    start.add_transition('ε', nfa.starting_state)
    start.add_transition('ε', end)

    for accepting_state in nfa.accepting_states:
        accepting_state.accepting = False
        accepting_state.add_transition('ε', end)

    combined_nfa = NFA(nfa.states + [start, end], start, [end])
    stack.append(combined_nfa)
    return state_counter + 2

def construct_nfa(postfix_regex):
    state_counter = 1
    stack = []

    for symbol in postfix_regex:
        if symbol.isalnum():
            state_counter = handle_character(stack, state_counter, symbol)
        elif symbol == '.':
            _,state_counter=handle_concatenation(stack, state_counter)
        elif symbol == '|':
            state_counter = handle_or(stack, state_counter)
        elif symbol == '*':
            state_counter = handle_kleene_star(stack, state_counter)
        elif symbol == '+':
            state_counter = handle_one_or_more(stack, state_counter)
        elif symbol == '?':
            state_counter = handle_zero_or_one(stack, state_counter)
        else:
            raise ValueError(f"Invalid symbol in postfix regex: {symbol}")

    return stack.pop()

def nfa_to_json(nfa):
    nfa_dict = {
        "startingState": nfa.starting_state.name,
    }

    for state in nfa.states:
        transitions = {symbol: [target.name for target in targets] for symbol, targets in state.transitions.items()}
        if state.accepting:
            transitions["isTerminatingState"] = True
        nfa_dict[state.name] = transitions

    return json.dumps(nfa_dict, indent=6, ensure_ascii=False)
def save_nfa_to_file(nfa, filename):
    nfa_json = nfa_to_json(nfa)
    with open(filename, 'w', encoding='utf-8') as file:
        file.write(nfa_json)



# Visualizing NFA

In [221]:
from graphviz import Digraph

def visualize_nfa(nfa):
    dot = Digraph(format='png')
    dot.attr(rankdir='LR')

    # Define the starting state
    dot.node(nfa.starting_state.name, shape='circle', style='filled', fillcolor='lightblue')

    # Define the states
    for state in nfa.states:
        shape = 'doublecircle' if state.accepting else 'circle'
        dot.node(state.name, shape=shape)

    # Add transitions
    for state in nfa.states:
        for symbol, targets in state.transitions.items():
            for target in targets:
                dot.edge(state.name, target.name, label=symbol)

    # Add an initial arrow
    dot.node('start', shape='none', label='')
    dot.edge('start', nfa.starting_state.name)

    # Save and render
    dot.render('nfa_visualization', view=True)




In [230]:
regex="(a*b)(b?a+)"
preprocessed_infix = preprocessing(regex)
print("\nPOSTFIX :")
postfix_regex=infix_to_postfix(preprocessed_infix)
print(postfix_regex)

nfa = construct_nfa(postfix_regex)

visualize_nfa(nfa)

save_nfa_to_file(nfa, "nfa_output.json")




Regex after replacing dot  : 
(a*b)(b?a+)

Regex after replacing square brackets  : 
(a*b)(b?a+)

Regex after adding concatination  : 
(a*b)(b?a+)

POSTFIX :
a*b.b?a+..


# Part 2

In [353]:
# from output json to input of part2
import json
def normalize_nfa(NFA):
    normalized_nfa = {}

    for state, transitions in NFA.items():
        # Preserve the starting state without modification
        if state == "startingState":
            normalized_nfa[state] = transitions
            continue

        normalized_nfa[state] = {}
        
        for symbol, target in transitions.items():
            if symbol == "isTerminatingState":
                normalized_nfa[state][symbol] = target
            else:
                # Ensure all targets are lists (even empty ones)
                if not isinstance(target, list):
                    target = [target] if target else []
                normalized_nfa[state][symbol] = target

        # Ensure states with no actions are represented as an empty list
        if not transitions:
            normalized_nfa[state] = {}

    return normalized_nfa

def read_json_file(file_path):
    try:
        with open(file_path, 'r',encoding='utf-8') as file:
            data = json.load(file)
            return data
    except FileNotFoundError:
        print(f"Error: The file {file_path} was not found.")
    except json.JSONDecodeError:
        print("Error: Failed to decode JSON from the file.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
#read json file in dict
nfa_dict=read_json_file("nfa_output.json")
normalize_nfa(nfa_dict)

{'startingState': 'S3',
 'S1': {'a': ['S2']},
 'S2': {'ε': ['S4', 'S1']},
 'S3': {'ε': ['S1']},
 'S4': {'ε': ['S5']},
 'S5': {'b': ['S6']},
 'S6': {'isTerminatingState': True}}

In [349]:
import json
import graphviz
class State:
    def __init__(self, name, elements):
        self.name = name
        self.elements = frozenset(elements)  # Ensure it's hashable

    def __hash__(self):
        return hash(self.elements)

    def __eq__(self, other):
        return self.elements == other.elements

    def __repr__(self):
        return f"State({self.name}, {sorted(self.elements)})"

class DFA:
    def __init__(self):
        self.states = {}  # Now stores State objects as values
        self.startingState = None
        self.transitions = {}
        self.acceptingStates = set()

    def add_state(self, state, is_accepting=False):
        self.states[state.name] = state
        if is_accepting:
            self.acceptingStates.add(state)

    def add_transition(self, from_state, to_state, symbol):
        if from_state.name not in self.transitions.keys():
            self.transitions[from_state.name]={}
        if symbol not in self.transitions[from_state.name].keys():
            self.transitions[from_state.name][symbol] = set()
        self.transitions[from_state.name][symbol].add(to_state.name)
        
        
    def get_transition(self, from_state):
        if from_state in self.transitions.keys():
            return self.transitions[from_state]
        return None

    def to_json(self, filename):
        dfa_json = {
            "startingState": self.startingState,
            "acceptingStates": [state.name for state in self.acceptingStates],
            "transitions": self.transitions
        }
        with open(filename, "w") as f:
            json.dump(dfa_json, f, indent=4)
        print(f"DFA saved to {filename}")
        
    def to_json_min(self, filename):
        dfa_json = {
            "startingState": self.startingState.name if self.startingState else None,
            "acceptingStates": [state.name for state in self.acceptingStates],
            "transitions": {
                from_state: {
                    symbol: [to_state for to_state in to_states]
                    for symbol, to_states in transitions.items()
                }
                for from_state, transitions in self.transitions.items()
            }
        }
        
        with open(filename, "w") as f:
            json.dump(dfa_json, f, indent=4)
        
        print(f"DFA saved to {filename}")

    def visualize(self, output_file="dfa_visualization"):
        dot = graphviz.Digraph(format='png')
        dot.attr(rankdir='LR')

        # Draw states
        for state in self.states.values():
            shape = "doublecircle" if state in self.acceptingStates else "circle"
            dot.node(state.name, label=state.name, shape=shape)

        # Draw starting arrow
        if self.startingState:
            dot.node("start", shape="point")
            dot.edge("start", self.startingState)

        # Draw transitions
        for from_state, transitions in self.transitions.items():
            for symbol, to_state in transitions.items():
                for state in to_state:
                    dot.edge(from_state, state, label=symbol)

        dot.render(output_file,view=True)
        print(f"DFA visualization saved to {output_file}.png")
    import graphviz

    def visualize_min(self, output_file="dfa_visualization"):
        dot = graphviz.Digraph(format='png')
        dot.attr(rankdir='LR')

        # Draw states
        for state in self.states.values():
            shape = "doublecircle" if state in self.acceptingStates else "circle"
            dot.node(state.name, label=state.name, shape=shape)

        # Ensure correct starting state
        if self.startingState:
            dot.node("Start", shape="point")  
            dot.edge("Start", self.startingState.name)  # Explicit label for clarity

        # Draw transitions
        for from_state, transitions in self.transitions.items():
            for symbol, to_state in transitions.items():
                for state in to_state:
                    dot.edge(from_state, state, label=symbol)

        dot.render(output_file, view=True)
        print(f"DFA visualization saved to {output_file}.png")

        
        
        
    def is_accepting_state(self, state):
        for s in self.acceptingStates:
            if s.name == state:
                return True
        return False

class NFAtoDFAConverter:
    def epsilon_closure(self, NFA, state):
        closure = {state}
        stack = [state]

        while stack:
            current = stack.pop()
            if "ε" in NFA[current]:
                for target in NFA[current]["ε"]:
                    if target not in closure:
                        closure.add(target)
                        stack.append(target)

        return closure

    def convert_to_DFA(self, NFA):
        DFA_machine = DFA()

        # Step 1: Compute the starting state (ε-closure of NFA start)
        start_closure = self.epsilon_closure(NFA, NFA["startingState"])
        start_state = State("S0", start_closure)
        DFA_machine.add_state(start_state, self.is_accepting_state(NFA, start_closure))
        DFA_machine.startingState = start_state.name

        # Step 2: Process unmarked states
        unmarked_states = [start_state]
        processed_states = set()

        while unmarked_states:
            current_state = unmarked_states.pop()
            if current_state in processed_states:
                continue
            processed_states.add(current_state)

            transitions_table = {}

            # Step 3: Iterate over all input symbols (exclude ε)
            alphabet = self.get_alphabet(NFA)

            for nfa_state in current_state.elements:
                for action in alphabet:
                    if action == 'ε':
                        continue
                    if action in NFA[nfa_state]:
                        next_states = set()
                        for next_state in NFA[nfa_state][action]:
                            next_states |= self.epsilon_closure(NFA, next_state)
                        if action not in transitions_table:
                            transitions_table[action] = next_states
                        else:
                            transitions_table[action] |= next_states

            # Step 4: Create new DFA states and transitions
            for action, next_states in transitions_table.items():
                if not next_states:
                    continue

                # Ensure consistent naming for new states
                
                next_state_name = f"S{len(DFA_machine.states)}"
                # print("//////////",next_states)
                # compact_name=""
                # for state in next_states:
                #     compact_name+=state +','
                # compact_name=compact_name[:-1]
                    
                next_state = State(next_state_name, next_states)

                # Check if state already exists
                if next_state.elements not in [s.elements for s in DFA_machine.states.values()]:
                    unmarked_states.append(next_state)
                    DFA_machine.add_state(next_state, self.is_accepting_state(NFA, next_states))
                else:
                    next_state = next(filter(lambda s: s.elements == next_state.elements, DFA_machine.states.values()))

                DFA_machine.add_transition(current_state, next_state, action)

        return DFA_machine

    def get_alphabet(self, NFA):
        alphabet = set()
        for transitions in NFA.values():
            if not isinstance(transitions, str):
                for symbol in transitions.keys():
                    if symbol != "ε" and symbol != "isTerminatingState":
                        alphabet.add(symbol)
        return alphabet

    def is_accepting_state(self, NFA, state_set):
        return any(NFA[s].get("isTerminatingState", False) for s in state_set)



In [302]:
converter = NFAtoDFAConverter()
dfa = converter.convert_to_DFA(nfa_dict)
dfa.visualize("dfa")
# dfa.to_json("dfa_out.json")
# Output DFA structure
# print("Starting State:", dfa.startingState)
# print("Accepting States:", dfa.acceptingStates)
# print("Transitions:")
for from_state, transitions in dfa.transitions.items():
    for symbol, to_state in transitions.items():
        print(f"{from_state} --{symbol}--> {to_state}")
        


DFA visualization saved to dfa.png
S0 --b--> {'S1'}
S0 --a--> {'S2'}
S2 --a--> {'S2'}
S2 --b--> {'S1'}
S1 --a--> {'S3'}
S1 --b--> {'S4'}
S4 --a--> {'S3'}
S3 --a--> {'S3'}


# Convert DFA to minimized DFA

In [316]:
class convertDFAToMinDFA:
    def convert_to_min_dfa(self, dfa: DFA):
        # Step 1: Separate accepting and non-accepting states
        P, Q = self.separate_acceptence(dfa)
        
        # Step 2: Create initial partition of states
        sets = [P, Q]
        new_states = True
        
        while new_states:
            new_states = False
            for state_set in sets:
                if len(state_set) > 1:
                    new_groups = {}
                    for state in state_set:
                        transition_table = {}
                        transitions = dfa.get_transition(state) or {}
                        
                        for symbol, target_states in transitions.items():
                            transition_table[symbol] = frozenset(target_states)
                        
                        transition_key = frozenset(transition_table.items())
                        if transition_key not in new_groups:
                            new_groups[transition_key] = {state}
                        else:
                            new_groups[transition_key].add(state)
                    
                    if len(new_groups) > 1:
                        sets.remove(state_set)
                        sets.extend(new_groups.values())
                        new_states = True
                        break
        
        sets = [s for s in sets if s]  # Remove empty sets
        print("Final Sets:", sets)
        
        final_dfa = self.create_min_DFA(dfa, sets)
        # final_dfa.startingState.name = "Start"
        return final_dfa
    
    def separate_acceptence(self, dfa: DFA):
        P, Q = set(), set()
        for state in dfa.states:
            if dfa.is_accepting_state(state):
                P.add(state)
            else:
                Q.add(state)
        return P, Q
    
    def create_min_DFA(self, dfa: DFA, sets) -> DFA:
        final_dfa = DFA()
        state_mapping = {}
        
        # Create new states
        for idx, states_set in enumerate(sets):
            new_state = State("S" + str(idx), frozenset(states_set))
            final_dfa.add_state(new_state)
            
            for state in states_set:
                state_mapping[state] = new_state
            
            if dfa.startingState in states_set:
                final_dfa.startingState = new_state
            
            if any(dfa.is_accepting_state(state) for state in states_set):
                final_dfa.acceptingStates.add(new_state)
        
        print("State Mapping:", state_mapping)
        
        # Add transitions to the minimized DFA
        for from_state, transitions in dfa.transitions.items():
            new_from_state = state_mapping[from_state]

            for symbol, to_states in transitions.items():
                for state in set(to_states):
                    new_to_state = state_mapping[state]
                    
                    existing_transitions = final_dfa.get_transition(new_from_state) or {}
                    print(existing_transitions)
                    if symbol not in existing_transitions or new_to_state not in existing_transitions[symbol]:
                        final_dfa.add_transition(new_from_state, new_to_state, symbol)
        
        return final_dfa


In [323]:
converter = NFAtoDFAConverter()
dfa = converter.convert_to_DFA(nfa_dict)
# print(dfa.transitions)
# dfa.to_json("dfa_out.json")
minDFA=convertDFAToMinDFA()
final_dfa=minDFA.convert_to_min_dfa(dfa)
final_dfa.visualize_min("dfa_min")
print(final_dfa.states)
# final_dfa.to_json("min_json.json")
# Output DFA structure
# print("Starting State:", dfa.startingState)
# print("Accepting States:", dfa.acceptingStates)
# print("Transitions:")
for from_state, transitions in final_dfa.transitions.items():
    for symbol, to_state in transitions.items():
        print(f"{from_state} --{symbol}--> {to_state}")
        


Final Sets: [{'S3'}, {'S4'}, {'S0', 'S2'}, {'S1'}]
State Mapping: {'S3': State(S0, ['S3']), 'S4': State(S1, ['S4']), 'S0': State(S2, ['S0', 'S2']), 'S2': State(S2, ['S0', 'S2']), 'S1': State(S3, ['S1'])}
{}
{}
{}
{}
{}
{}
{}
{}
DFA visualization saved to dfa_min.png
{'S0': State(S0, ['S3']), 'S1': State(S1, ['S4']), 'S2': State(S2, ['S0', 'S2']), 'S3': State(S3, ['S1'])}
S2 --b--> {'S3'}
S2 --a--> {'S2'}
S3 --a--> {'S0'}
S3 --b--> {'S1'}
S1 --a--> {'S0'}
S0 --a--> {'S0'}


In [355]:
# final cell 
regex="a+b"
preprocessed_infix = preprocessing(regex)
print("\nPOSTFIX :")
postfix_regex=infix_to_postfix(preprocessed_infix)
print(postfix_regex)

nfa = construct_nfa(postfix_regex)

visualize_nfa(nfa)

save_nfa_to_file(nfa, "nfa_output.json")
# from NFA to DFA
nfa_dict=read_json_file("nfa_output.json")
converter = NFAtoDFAConverter()
dfa = converter.convert_to_DFA(nfa_dict)
# dfa.visualize("dfa")

#from DFA to minDFA
minDFA=convertDFAToMinDFA()
final_dfa=minDFA.convert_to_min_dfa(dfa)
final_dfa.visualize_min("dfa_min")
final_dfa.to_json_min("dfa_min.json")


Regex after replacing dot  : 
a+b

Regex after replacing square brackets  : 
a+b

Regex after adding concatination  : 
a+b

POSTFIX :
a+b.
Final Sets: [{'S2'}, {'S0'}, {'S1'}]
State Mapping: {'S2': State(S0, ['S2']), 'S0': State(S1, ['S0']), 'S1': State(S2, ['S1'])}
{}
{}
{}
DFA visualization saved to dfa_min.png
DFA saved to dfa_min.json
